In [101]:
import joblib
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
# Load the dataset
df = pd.read_csv('data.csv')
# Convert class names to categorical codes
df['class_name'] = df['class_name'].astype('category')
df['class_label'] = df['class_name'].cat.codes

# Prepare features and labels
X = df.iloc[:, 1:29]  # Assuming you want columns from index 1 to 27
y = df['class_label']

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.01, random_state=42)

# Feature scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Save the scaler
joblib.dump(scaler, 'scaler.pkl')


# K-Nearest Neighbors (KNN)
knn = KNeighborsClassifier(n_neighbors=10)
knn.fit(X_train_scaled, y_train)
knn_predictions = knn.predict(X_test_scaled)
knn_accuracy = accuracy_score(y_test, knn_predictions)
print(f'KNN Accuracy: {knn_accuracy:.2f}')

# Save the KNN model
joblib.dump(knn, 'knn.pkl')


# Support Vector Classifier (SVC)
svm = SVC(kernel='linear', C=100)
svm.fit(X_train_scaled, y_train)
svm_predictions = svm.predict(X_test_scaled)
svm_accuracy = accuracy_score(y_test, svm_predictions)
print(f'SVM Accuracy: {svm_accuracy:.2f}')
# Save the SVM model
joblib.dump(svm, 'svm.pkl')


# Random Forest Hyperparameter Tuning
rf_params = {'n_estimators': [50, 100, 200], 'max_depth': [None, 10, 20, 30]}
rf_grid = GridSearchCV(RandomForestClassifier(random_state=42), rf_params, cv=5)
rf_grid.fit(X_train_scaled, y_train)
print(f'Best RF Params: {rf_grid.best_params_}')
print(f'Best RF Cross-Validation Score: {rf_grid.best_score_}')

# Save the best Random Forest model
joblib.dump(rf_grid.best_estimator_, 'rf.pkl')

KNN Accuracy: 0.40
SVM Accuracy: 0.90
Best RF Params: {'max_depth': 10, 'n_estimators': 100}
Best RF Cross-Validation Score: 0.6464646464646464


['rf.pkl']

In [1]:
import librosa
import numpy as np
import joblib

# Load the model and scaler
model = joblib.load('rf.pkl')
scaler = joblib.load('scaler.pkl')

def getmetadata(filename):
    y, sr = librosa.load(filename)
    
    # Use keyword arguments for onset_strength
    onset_env = librosa.onset.onset_strength(y=y, sr=sr)
    tempo = librosa.beat.tempo(onset_envelope=onset_env, sr=sr)
    
    # Use harmonic-percussive separation
    y_harmonic, y_percussive = librosa.effects.hpss(y)
    
    # Beat tracking
    tempo, beat_frames = librosa.beat.beat_track(y=y_percussive, sr=sr)
    
    # Feature extraction
    chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
    rmse = librosa.feature.rms(y=y)
    spec_centroid = librosa.feature.spectral_centroid(y=y, sr=sr)[0]
    spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
    spec_rolloff = librosa.feature.spectral_rolloff(y=y+0.01, sr=sr)[0]
    zero_crossing = librosa.feature.zero_crossing_rate(y=y)
    mfcc = librosa.feature.mfcc(y=y, sr=sr)
    
    # Metadata dictionary
    metadata_dict = {
        'tempo': np.mean(tempo),
        'beats': np.mean(beat_frames),
        'chroma_stft': np.mean(chroma_stft),
        'rmse': np.mean(rmse),
        'spectral_centroid': np.mean(spec_centroid),
        'spectral_bandwidth': np.mean(spec_bw),
        'rolloff': np.mean(spec_rolloff),
        'zero_crossing_rates': np.mean(zero_crossing)
    }
    
    for i in range(1, 21):
        if i <= mfcc.shape[0]:  # Check if the index is within the bounds of mfcc
            metadata_dict[f'mfcc{i}'] = np.mean(mfcc[i-1])
    
    return metadata_dict


In [2]:
genre_mapping = {0: 'blues', 1: 'classical', 2: 'country', 3: 'disco', 4: 'hiphop', 5: 'jazz', 6: 'metal', 7: 'pop', 8: 'reggae', 9: 'rock'}

def find_genre(file_path):
    mtdt = getmetadata(file_path)
    mtdt = np.array(list(mtdt.values()))
    mtdt.reshape(-1,1)
    mtdt_scaled = scaler.transform([mtdt])
    pred_genre = model.predict(mtdt_scaled) 
    return genre_mapping[pred_genre[0]]

In [3]:
import gradio as gr
interface = gr.Interface(
    fn=find_genre,  # The function to wrap
    inputs=gr.Audio(type="filepath"),  # Input type
    outputs="text",  # Output type
    title="Music Genre Classification",  # Title of the interface
    description="Upload an audio file to classify its music genre."  # Description
)

# Launch the Gradio interface
interface.launch()

C:\Users\Chaithanya\AppData\Roaming\Python\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


C:\Users\Chaithanya\AppData\Local\Temp\ipykernel_30292\3673371580.py:14: FutureWarning: librosa.beat.tempo
	This function was moved to 'librosa.feature.rhythm.tempo' in librosa version 0.10.0.
	This alias will be removed in librosa version 1.0.
  tempo = librosa.beat.tempo(onset_envelope=onset_env, sr=sr)
C:\Users\Chaithanya\AppData\Roaming\Python\Python311\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
